# 답변의 형식을 컨트롤하는 방법

이 노트북에서는 LangChain을 사용하여 LLM의 출력 형식을 제어하는 다양한 방법을 알아보겠습니다.

## 1. LLM 설정

먼저 기본적인 LLM을 설정합니다.


In [1]:
from langchain_ollama import ChatOllama

# Ollama를 이용한 로컬 LLM 설정

llm = ChatOllama(model="gemma3:12b")


## 2. 문자열 출력 파서 사용

StrOutputParser를 사용하여 LLM의 출력을 단순 문자열로 변환할 수 있습니다.


In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 명시적인 지시사항이 포함된 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the city only",
    input_variables=["country"],
)
# 템플릿에 변수 값을 주입
prompt = prompt_template.invoke({"country": "France"})

print(prompt)

# LLM에 프롬프트 전달
ai_message = llm.invoke(prompt_template.invoke({"country": "France"}))

print(ai_message)

# 문자열 출력 파서를 사용하여 응답을 단순 문자열로 변환
output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "France"})))

text='What is the capital of France? Return the name of the city only'
content='Paris.' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-02-02T14:22:47.349943Z', 'done': True, 'done_reason': 'stop', 'total_duration': 183572167, 'load_duration': 31401625, 'prompt_eval_count': 39, 'prompt_eval_duration': 108000000, 'eval_count': 3, 'eval_duration': 43000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-f9bd3685-abd4-4ed3-bdca-f156e680d41b-0' usage_metadata={'input_tokens': 39, 'output_tokens': 3, 'total_tokens': 42}


## 3. 응답 타입 확인

LLM의 응답이 어떤 형식으로 반환되는지 확인해봅니다.


In [15]:
# AI 메시지의 content 속성이 문자열 타입임을 확인

type(ai_message.content)

str

In [10]:
# 파싱된 응답 확인
answer

'Paris'

## 4. 구조화된 출력 사용

Pydantic 모델을 사용하여 LLM의 출력을 구조화된 형식으로 받을 수 있습니다.


In [19]:
from pydantic import BaseModel, Field

# 국가 정보를 담을 Pydantic 모델 정의
class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")


# LLM에 구조화된 출력 형식 지정
structued_llm = llm.with_structured_output(CountryDetail)

In [20]:
from langchain_core.output_parsers import JsonOutputParser

# JSON 형식으로 응답을 요청하는 프롬프트 템플릿
country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}:
    - Capital
    - Population
    - Language
    - Currency

    return it in JSON format. and return the JSON dictionry only 
    """,
    input_variables=["country"],
)

country_detail_prompt.invoke({"country": "France"})


# output_parser = JsonOutputParser()
# 구조화된 LLM으로 응답 받기
json_ai_message = structued_llm.invoke(country_detail_prompt.invoke({"country": "France"}))
# output_parser.invoke(json_ai_message)

In [21]:
# 구조화된 응답 확인
(json_ai_message)

CountryDetail(capital='Paris', population=65275746, language='French', currency='Euro')

In [25]:
# Pydantic 모델의 특정 필드 접근
json_ai_message.model_dump()["capital"]

'Paris'